# Data Engineering Interview Exercise Notebook
 
---
 
## 📖 Scenario
 
You are a Data Engineer at an international e-commerce company. Your task is to build a data pipeline that processes daily transaction data, enriches it with customer information, and produces an aggregated report to identify top-spending customers.
 
You have been provided with two CSV files:
- `transactions.csv`: Transaction details (transaction_id, customer_id, product_id, quantity, price, date).
- `customers.csv`: Customer details (customer_id, name, email, join_date).
 
---

## 🛠️ Step-by-Step Instructions
 
Complete each step below. You may choose either Pandas or PySpark for this exercise.
 
### ⚙️ Step 1: Data Ingestion
- Load both CSV files into DataFrames.
 
**Hint**: 
- Pandas: `pd.read_csv()`
- PySpark: `spark.read.csv()`

In [2]:
# Import for Pandas users
import pandas as pd

In [1]:
# Imports for PySpark users
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

spark = SparkSession.builder.appName("EcommerceETL")

In [2]:
# Start Spark session
spark = SparkSession.builder.appName("InterviewApp").getOrCreate()

# Example DataFrame
df = spark.createDataFrame([
    (1, "Alice", 100),
    (2, "Bob", 200)
], ["id", "name", "amount"])

df.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/31 15:44:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is supported only if a security manager is allowed
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:347)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:577)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2416)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2416)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:329)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:501)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:485)
	a

In [5]:
# Pandas
transactions = pd.read_csv('../data/transactions.csv')
customers = pd.read_csv('../data/customers.csv')
transactions.head()


,transaction_id,customer_id,product_id,quantity,price,date
0,1001,2001,3001,2,19.99,2024-01-01
1,1002,2002,3003,1,49.99,2024-01-01
2,1003,2001,3002,3,9.99,2024-01-02
3,1004,2003,3001,1,19.99,2024-01-02
4,1005,2004,3004,2,29.99,2024-01-03


In [6]:
customers.head()

,customer_id,name,email,join_date
0,2001,John Doe,johndoe@example.com,2023-06-15
1,2002,Jane Smith,janesmith@example.com,2023-07-20
2,2003,Bob Johnson,bobjohnson@example.com,2023-08-10
3,2004,Alice Williams,alicewilliams@example.com,2023-09-05
4,2005,Michael Brown,michaelbrown@example.com,2023-10-12


In [11]:
# PySpark
transactions_spark = spark.read.csv('..data/transactions.csv', header=True, inferSchema=True)
customers_spark = spark.read.csv('../data/customers.csv', header=True, inferSchema=True)

AttributeError: 'Builder' object has no attribute 'read'

### 🧹 Step 2: Data Cleaning
- Remove duplicates from both datasets.
 
**Hint**: 
- Pandas: `drop_duplicates()`
- PySpark: `dropDuplicates()`

In [4]:
# Pandas
transactions = transactions.drop_duplicates(subset='transaction_id')
customers = customers.drop_duplicates(subset='customer_id')

In [ ]:
# PySpark
transactions_spark = transactions_spark.dropDuplicates(['transaction_id'])
customers_spark = customers_spark.dropDuplicates(['customer_id'])

### 🔗 Step 3: Data Joining
- Join transaction data with customer data on `customer_id`.
 
**Hint**:
- Pandas: `merge()`
- PySpark: `.join()`

In [5]:
# Pandas
joined_df = transactions.merge(customers, on='customer_id', how='inner')

In [ ]:
# PySpark
joined_spark_df = transactions_spark.join(customers_spark, 'customer_id', 'inner')

In [ ]:
# Spark SQL
transactions_spark.createOrReplaceTempView("transactions")
customers_spark.createOrReplaceTempView("customers")
 
joined_sql_df = spark.sql("""
SELECT t.*, c.name, c.email, c.join_date
FROM transactions t
INNER JOIN customers c
ON t.customer_id = c.customer_id
""")

### 📊 Step 4: Data Aggregation
- Calculate the total amount spent (`quantity * price`) per customer.
- Find the top 5 customers based on total spending.
 
**Hint**:
- Pandas: Use `groupby()` and aggregate with `.sum()`
- PySpark: Use `groupBy()` and aggregation functions (`sum()`)

In [6]:
# Pandas
joined_df['total_spent'] = joined_df['quantity'] * joined_df['price']
top_customers = joined_df.groupby(['customer_id', 'name'])['total_spent'].sum().reset_index().sort_values(by='total_spent', ascending=False).head(5)

In [ ]:
top_customers.head()

,customer_id,name,total_spent
4,2005,Michael Brown,249.95
2,2003,Bob Johnson,169.96
3,2004,Alice Williams,99.96
0,2001,John Doe,99.94
1,2002,Jane Smith,59.98


In [ ]:
# PySpark
joined_spark_df = joined_spark_df.withColumn('total_spent', col('quantity') * col('price'))
top_customers_spark = (joined_spark_df.groupBy('customer_id', 'name')
                       .agg(sum('total_spent').alias('total_spent'))
                       .orderBy(col('total_spent').desc())
                       .limit(5))

In [ ]:
# Spark SQL
joined_sql_df.createOrReplaceTempView("joined_data")
top_customers_sql = spark.sql("""
SELECT customer_id, name, SUM(quantity * price) AS total_spent
FROM joined_data
GROUP BY customer_id, name
ORDER BY total_spent DESC
LIMIT 5
""")

### 📁 Step 5: Export Results
- Export the aggregated results as a CSV file named `top_customers.csv`.
 
**Hint**:
- Pandas: `to_csv()`
- PySpark: `write.csv()`

In [7]:
# Pandas
top_customers.to_csv('top_customers.csv', index=False)

In [ ]:
# PySpark
top_customers_spark.write.csv('top_customers.csv', header=True)

In [ ]:
# --------------------------- Scenario-based Questions & Answers ---------------------------
 
# (Q1) Warehouse vs. Data Lake
# Answer:
# Use a data warehouse for structured data, fast analytics, and BI use-cases. 
# Use a data lake for flexible storage of unstructured/semi-structured data and exploratory analytics.
 
# (Q2) Cloud Storage (AWS vs GCP)
# Answer:
# AWS: S3 for storage, Glue or EMR for ETL, Redshift/Athena for querying.
# GCP: Cloud Storage for storage, Dataflow/Dataproc for ETL, BigQuery for querying.
 
# (Q3) Batch vs. Streaming
# Answer:
# Streaming data pipelines would use Kafka/Kinesis/PubSub for ingestion, with Spark Streaming or Flink for real-time processing.
 
# (Q4) Handling Large Datasets
# Answer:
# Optimize joins and aggregations through data partitioning/bucketing, distributed processing frameworks (Spark), and caching mechanisms.
 
# (Q5) Failure Handling & Idempotency
# Answer:
# Ensure idempotency by avoiding side effects, checkpointing pipelines, and having unique keys. Use retry mechanisms and thorough logging for reliability.